# Membangun Agen AI dengan Memori Persisten menggunakan Cognee

Notebook ini menunjukkan cara membangun agen AI cerdas dengan kemampuan memori yang canggih menggunakan [**cognee**](https://www.cognee.ai/) - memori AI open source yang menggabungkan grafik pengetahuan, pencarian semantik, dan manajemen sesi untuk menciptakan sistem AI yang sadar konteks.

## 🎯 Tujuan Pembelajaran

Di akhir tutorial ini, Anda akan memahami cara:
- **Membangun Grafik Pengetahuan yang Didukung oleh Embedding**: Mengubah teks tidak terstruktur menjadi pengetahuan yang terstruktur dan dapat di-query
- **Mengimplementasikan Memori Sesi**: Membuat percakapan multi-putaran dengan retensi konteks otomatis
- **Menyimpan Percakapan**: Secara opsional menyimpan interaksi penting dalam memori jangka panjang untuk referensi di masa depan
- **Melakukan Query Menggunakan Bahasa Alami**: Mengakses dan memanfaatkan konteks historis dalam percakapan baru
- **Memvisualisasikan Memori**: Mengeksplorasi hubungan dalam grafik pengetahuan agen Anda


## 🏗️ Apa yang Akan Anda Bangun

Dalam tutorial ini, kita akan membuat **Asisten Koding** dengan memori yang persisten yang:

### 1. **Konstruksi Basis Pengetahuan**
   - Mengambil informasi profil dan keahlian pengembang
   - Memproses prinsip dan praktik terbaik pemrograman Python
   - Menyimpan percakapan historis antara pengembang dan asisten AI

### 2. **Percakapan yang Sadar Konteks**
   - Mempertahankan konteks di antara beberapa pertanyaan dalam sesi yang sama
   - Secara otomatis menyimpan setiap pasangan pertanyaan/jawaban untuk pengambilan yang efisien
   - Memberikan respons yang koheren dan kontekstual berdasarkan riwayat percakapan

### 3. **Memori Jangka Panjang**
   - Menyimpan percakapan penting ke dalam memori jangka panjang
   - Mengambil memori yang relevan dari basis pengetahuan dan sesi sebelumnya untuk menginformasikan interaksi baru
   - Membangun basis pengetahuan yang terus berkembang dan semakin baik seiring waktu

### 4. **Pengambilan Memori yang Cerdas**
   - Menggunakan pencarian semantik berbasis graf untuk menemukan informasi relevan di seluruh pengetahuan yang tersimpan
   - Menyaring pencarian berdasarkan subkelompok data (informasi pengembang vs. prinsip)
   - Menggabungkan berbagai sumber data untuk memberikan jawaban yang komprehensif


## 📋 Prasyarat & Pengaturan

### Persyaratan Sistem

Sebelum memulai, pastikan Anda memiliki:

1. **Lingkungan Python**
   - Python 3.9 atau lebih tinggi
   - Lingkungan virtual (disarankan)
   
2. **Redis Cache** (Diperlukan untuk Manajemen Sesi)
   - Redis Lokal: `docker run -d -p 6379:6379 redis`
   - Atau gunakan layanan Redis yang dikelola
   
3. **Akses API LLM**
   - Kunci API OpenAI atau penyedia lainnya (lihat [dokumentasi](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfigurasi Basis Data**
   - Tidak diperlukan konfigurasi secara default. Cognee menggunakan basis data berbasis file (LanceDB dan Kuzu)
   - Opsional, Anda dapat mengatur Azure AI Search sebagai penyimpanan vektor (lihat [dokumentasi](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfigurasi Lingkungan

Buat file `.env` di direktori proyek Anda dengan variabel berikut:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Memahami Arsitektur Memori Cognee

### Cara Kerja Cognee

Cognee menyediakan sistem memori canggih yang melampaui penyimpanan key-value sederhana:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Komponen Utama:

1. **Knowledge Graph**: Menyimpan entitas, hubungan, dan koneksi semantik
2. **Vector Embeddings**: Memungkinkan pencarian semantik di seluruh informasi yang disimpan
3. **Session Cache**: Mempertahankan konteks percakapan dalam dan antar sesi
4. **NodeSets**: Mengorganisasi data ke dalam kategori logis untuk pengambilan data yang terarah

### Jenis Memori dalam Tutorial Ini:

- **Persistent Memory**: Penyimpanan jangka panjang dalam knowledge graph
- **Session Memory**: Konteks percakapan sementara dalam cache Redis
- **Semantic Memory**: Pencarian berbasis kesamaan vektor di seluruh data


## 📦 Pasang Paket yang Diperlukan

Pasang Cognee dengan dukungan Redis untuk pengelolaan sesi:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inisialisasi Lingkungan dan Muat Library

Pastikan:
1. Redis sedang berjalan (misalnya, melalui Docker: `docker run -d -p 6379:6379 redis`)
2. Variabel lingkungan telah diatur sebelum mengimpor modul cache
3. Jika diperlukan, restart kernel dan jalankan sel secara berurutan

Sel berikut akan:
1. Memuat variabel lingkungan dari `.env`
2. Mengonfigurasi Cognee dengan pengaturan LLM Anda
3. Mengaktifkan caching untuk manajemen sesi
4. Memvalidasi semua komponen terhubung dengan benar


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurasi Direktori Penyimpanan

Cognee menggunakan dua direktori terpisah untuk operasinya:
- **Data Root**: Menyimpan dokumen yang diunggah dan data yang telah diproses
- **System Root**: Berisi basis data grafik pengetahuan dan metadata sistem

Kita akan membuat direktori terpisah untuk tutorial ini sebagai berikut:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Atur Ulang Status Memori

Sebelum kita mulai membangun sistem memori, mari pastikan kita memulai dari awal.

> 💡 **Tip**: Anda dapat melewati langkah ini jika ingin mempertahankan memori yang ada dari penggunaan notebook sebelumnya.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Bagian 1: Membangun Basis Pengetahuan

### Sumber Data untuk Asisten Pengembang Kami

Kami akan mengumpulkan tiga jenis data untuk membuat basis pengetahuan yang komprehensif:

1. **Profil Pengembang**: Keahlian pribadi dan latar belakang teknis
2. **Praktik Terbaik Python**: Zen of Python dengan panduan praktis
3. **Percakapan Historis**: Sesi tanya jawab sebelumnya antara pengembang dan asisten AI

Data yang beragam ini memungkinkan agen kami untuk:
- Memahami konteks teknis pengguna
- Menerapkan praktik terbaik dalam rekomendasi
- Belajar dari interaksi sukses sebelumnya


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Proses Data menjadi Knowledge Graph

Sekarang kita akan mengubah teks mentah menjadi memori yang terstruktur. Proses ini:

1. **Menambahkan data ke NodeSets**: Mengorganisasi informasi ke dalam kategori logis
   - `developer_data`: Profil pengembang dan percakapan
   - `principles_data`: Praktik terbaik dan panduan Python

2. **Menjalankan Cognify Pipeline**: Mengekstrak entitas, hubungan, dan membuat embeddings
   - Mengidentifikasi konsep utama
   - Membuat koneksi semantik antara informasi yang terkait
   - Menghasilkan vector embeddings

Proses ini mungkin memerlukan beberapa saat karena LLM memproses teks dan membangun struktur graf:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualisasikan Grafik Pengetahuan

Mari kita jelajahi struktur grafik pengetahuan kita. Visualisasi ini menunjukkan:
- **Node**: Entitas yang diekstrak dari teks (konsep, teknologi, orang)
- **Edge**: Hubungan dan koneksi antar entitas
- **Cluster**: Konsep terkait yang dikelompokkan berdasarkan kesamaan semantik

Buka file HTML yang dihasilkan di browser Anda untuk menjelajahi grafik secara interaktif:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Perkaya Memori dengan Memify

Fungsi `memify()` menganalisis grafik pengetahuan dan menghasilkan aturan cerdas tentang data. Proses ini:
- Mengidentifikasi pola dan praktik terbaik
- Membuat panduan yang dapat diterapkan berdasarkan konten
- Menetapkan hubungan antara berbagai area pengetahuan

Aturan-aturan ini membantu agen membuat keputusan yang lebih terinformasi saat menjawab pertanyaan. Menangkap visualisasi kedua membantu Anda membandingkan bagaimana grafik menjadi lebih padat setelah diperkaya.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Bagian 2: Pengambilan Memori Cerdas

### Demonstrasi 1: Integrasi Pengetahuan Antar-Dokumen

Sekarang grafik pengetahuan kita telah dibuat, mari kita uji bagaimana Cognee menggabungkan informasi dari berbagai sumber untuk menjawab pertanyaan yang kompleks.

Pertanyaan pertama menunjukkan:
- **Pemahaman semantik**: Menemukan konsep yang relevan meskipun tidak disebutkan secara eksplisit
- **Referensi silang**: Menggabungkan profil pengembang dengan prinsip Python
- **Penalaran kontekstual**: Menerapkan praktik terbaik pada implementasi tertentu

### Demonstrasi 2: Pencarian Tersaring dengan NodeSets

Pertanyaan kedua menunjukkan cara menargetkan subset tertentu dari grafik pengetahuan:
- Menggunakan parameter `node_name` untuk mencari hanya dalam `principles_data`
- Memberikan jawaban yang terfokus dari domain pengetahuan tertentu
- Berguna saat Anda membutuhkan informasi spesifik domain


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Bagian 3: Pengaturan Manajemen Sesi

### Mengaktifkan Memori Percakapan

Manajemen sesi sangat penting untuk menjaga konteks di antara beberapa interaksi. Di sini kita akan:

1. **Inisialisasi Konteks Pengguna**: Membuat atau mengambil profil pengguna untuk pelacakan sesi
2. **Konfigurasi Mesin Cache**: Terhubung ke Redis untuk menyimpan riwayat percakapan
3. **Mengaktifkan Variabel Sesi**: Menyiapkan variabel konteks yang bertahan di antara kueri

> ⚠️ **Penting**: Ini membutuhkan Redis berjalan dan `CACHING=true` di lingkungan Anda


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Fungsi Pembantu: Melihat Riwayat Sesi

Fungsi utilitas ini memungkinkan kita untuk memeriksa riwayat percakapan yang disimpan di Redis. Ini berguna untuk:
- Debugging pengelolaan sesi
- Memastikan bahwa percakapan disimpan dalam cache
- Memahami konteks apa yang tersedia untuk agen


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesi 1: Lab Dukungan Async — Pertanyaan Pertama

Mulai sesi `async-support-lab` dengan menanyakan pola asyncio yang ramah telemetri untuk sebuah web scraper besar. Grafik sudah mengetahui tentang asyncio, aiohttp, dan praktik pemantauan, jadi responsnya harus mencerminkan percakapan sebelumnya sambil menyesuaikan jawaban dengan pertanyaan baru.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Periksa Memori Sesi 1 Setelah Pertukaran Pertama

Menjalankan `show_history(session_1)` segera setelah pertanyaan awal mengonfirmasi bahwa Cognee menulis baik prompt maupun penyelesaian ke Redis. Anda seharusnya melihat satu entri dengan panduan konkuren.


In [ ]:
await show_history(session_1)

## Sesi 1: Tindak Lanjut pada Model Data

Selanjutnya kita bertanya, "Kapan saya harus memilih dataclasses dibandingkan Pydantic?" menggunakan ID sesi yang sama. Cognee seharusnya menggabungkan prinsip-prinsip Python ditambah percakapan FastAPI sebelumnya untuk memberikan saran yang mendalam—menunjukkan bahwa konteks tetap berlanjut dalam sesi yang diberi nama.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Konfirmasi Riwayat Sesi 1 Berisi Kedua Giliran

Panggilan `show_history(session_1)` lainnya seharusnya menunjukkan dua entri Tanya Jawab. Ini sesuai dengan langkah "pemutaran ulang memori" di lab Mem0 dan membuktikan bahwa giliran tambahan memperpanjang transkrip yang sama.


In [ ]:
await show_history(session_1)

## Sesi 2: Thread Tinjauan Desain — Sesi Baru

Untuk menunjukkan isolasi antara thread, kami memulai `design-review-session` dan meminta panduan pencatatan untuk tinjauan insiden. Meskipun basis pengetahuan dasarnya sama, ID sesi yang baru menjaga transkrip tetap terpisah.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Tinjauan Sesi 2 Sejarah

`show_history(session_2)` seharusnya hanya mencantumkan pasangan prompt/respons tinjauan desain. Bandingkan dengan Sesi 1 untuk menyoroti bagaimana Cognee menjaga transkrip independen sambil menggunakan kembali grafik pengetahuan bersama.


In [ ]:
await show_history(session_2)

## Ringkasan

Selamat! Anda baru saja memberikan asisten pemrograman Anda lapisan memori jangka panjang yang didukung oleh Cognee.

Dalam tutorial ini, Anda mengambil konten pengembang mentah (kode, dokumen, obrolan) dan mengubahnya menjadi grafik + memori vektor yang dapat dicari, dianalisis, dan terus ditingkatkan oleh agen Anda.

Apa yang Telah Anda Pelajari

1. **Dari teks mentah ke memori AI**: Bagaimana Cognee mengolah data tidak terstruktur dan mengubahnya menjadi memori cerdas yang dapat dicari menggunakan arsitektur gabungan vektor + grafik pengetahuan.

2. **Peningkatan grafik dengan memify**: Bagaimana melampaui pembuatan grafik dasar dan menggunakan memify untuk menambahkan fakta turunan dan hubungan yang lebih kaya di atas grafik yang sudah ada.

3. **Berbagai strategi pencarian**: Bagaimana melakukan pencarian memori dengan berbagai jenis pencarian (Q&A berbasis grafik, penyelesaian gaya RAG, wawasan, potongan mentah, pencarian kode, dll.) tergantung pada kebutuhan agen Anda.

4. **Eksplorasi visual**: Bagaimana memeriksa dan memperbaiki apa yang dibangun Cognee menggunakan visualisasi grafik dan UI Cognee, sehingga Anda benar-benar dapat melihat bagaimana pengetahuan disusun.

5. **Memori yang sadar sesi**: Bagaimana menggabungkan konteks per sesi dengan memori semantik yang persisten sehingga agen dapat mengingat antar sesi tanpa membocorkan informasi antar pengguna.


## Poin Penting
1. Memori sebagai Knowledge Graph yang didukung oleh Embeddings

    - **Pemahaman terstruktur**: Cognee menggabungkan penyimpanan vektor dan penyimpanan graf sehingga data Anda dapat dicari berdasarkan makna dan terhubung melalui hubungan. Secara default, Cognee menggunakan database berbasis file (LanceDB untuk penyimpanan vektor, Kuzu untuk database graf).

    - **Pengambilan yang sadar hubungan**: Jawaban tidak hanya didasarkan pada "teks serupa," tetapi juga pada bagaimana entitas saling berhubungan.

    - **Memori yang hidup**: Lapisan memori berkembang, tumbuh, dan tetap dapat diakses sebagai satu graf yang terhubung.

2. Mode Pencarian & Penalaran
    - **Pengambilan hibrida**: Pencarian menggabungkan kesamaan vektor, struktur graf, dan penalaran LLM, mulai dari pencarian potongan mentah hingga jawaban pertanyaan yang sadar graf.

    - **Sesuaikan mode dengan tugas**: Gunakan mode gaya penyelesaian ketika Anda menginginkan jawaban dalam bahasa alami, dan mode potongan/ringkasan/graf ketika agen Anda membutuhkan konteks mentah atau untuk mendorong penalarannya sendiri.

3. Agen yang Dipersonalisasi dan Sadar Sesi
    - **Konteks sesi + memori jangka panjang**: Cognee memisahkan konteks "thread" jangka pendek dari memori jangka panjang yang berfokus pada pengguna atau organisasi.

## Aplikasi Dunia Nyata

1. **Agen AI Vertikal**

    Gunakan pola dari notebook ini untuk mendukung co-pilot cerdas domain yang berada di atas Cognee sebagai inti pengambilan dan penalarannya:

- **Co-pilot pengembang**: Tinjauan kode, analisis insiden, dan asisten arsitektur yang menjelajahi kode, API, dokumen desain, dan tiket sebagai satu graf memori.

- **Co-pilot untuk pelanggan**: Agen dukungan atau keberhasilan yang menarik dari dokumen produk, FAQ, catatan CRM, dan tiket sebelumnya dengan pengambilan yang sadar graf dan jawaban yang dikutip.

- **Co-pilot ahli internal**: Asisten kebijakan, hukum, atau keamanan yang menalar berdasarkan aturan, pedoman, dan keputusan historis yang saling terhubung, bukan PDF yang terisolasi.

    Cognee secara eksplisit diposisikan sebagai memori yang persisten dan akurat untuk agen AI, menyediakan knowledge graph yang hidup yang mendukung agen Anda dan menggantikan kombinasi ad-hoc dari penyimpanan vektor dan kode graf khusus.

2. **Menyatukan Data Silo menjadi Satu Memori**

    Pendekatan yang sama juga membantu Anda membangun lapisan memori terpadu di berbagai sumber yang tersebar:

- **Dari silo ke satu graf**: Masukkan data terstruktur (misalnya, database) dan tidak terstruktur (misalnya, dokumen, obrolan) ke dalam satu graf yang didukung oleh embeddings, daripada indeks terpisah untuk setiap sistem.

- **Penalaran lintas sumber dengan kutipan**: Jalankan penalaran multi-langkah di atas semuanya—“gabungkan” log, metrik, dan dokumen melalui graf—dan tetap kembalikan jawaban yang didasarkan dengan asal-usul yang jelas.

- **Pusat pengetahuan**: Untuk domain seperti perbankan atau pendidikan, Cognee sudah digunakan untuk menyatukan PDF, sistem internal, dan data aplikasi ke dalam satu knowledge graph dengan vektor sehingga agen dapat menjawab pertanyaan dengan konteks yang tepat dan dikutip.

## Langkah Selanjutnya

Anda telah mengimplementasikan loop memori inti. Berikut adalah pengembangan alami yang dapat Anda coba sendiri (lihat [dokumentasi Cognee](https://docs.cognee.ai/) untuk detailnya):

1. **Bereksperimen dengan kesadaran temporal**: Aktifkan temporal cognify untuk mengekstrak peristiwa dan stempel waktu dari teks.

2. **Perkenalkan penalaran berbasis ontologi**: Definisikan ontologi OWL untuk domain Anda. Gunakan dukungan ontologi Cognee sehingga entitas dan hubungan yang diekstraksi didasarkan pada skema tersebut, meningkatkan kualitas graf dan jawaban spesifik domain.

3. **Tambahkan loop umpan balik**: Biarkan Cognee menyesuaikan bobot tepi graf dari umpan balik pengguna nyata, sehingga pengambilan data meningkat seiring waktu daripada tetap statis.

4. **Sesuaikan untuk personalisasi & perilaku sesi**: Gunakan ID pengguna, penyewa, dan dataset untuk memberikan setiap orang atau tim pandangan mereka sendiri atas mesin memori bersama.

5. **Skalakan ke agen yang lebih kompleks**: Hubungkan Cognee ke kerangka kerja agen untuk membangun sistem multi-agen yang semuanya berbagi lapisan memori yang sama. *Microsoft Agent Framework x plugin Cognee akan segera hadir.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
